# RoBERTa model za klasifikaciju teksta pomoću Trainer klase


In [2]:
pip install datasets

In [3]:
import numpy as np
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from datasets import load_dataset

In [4]:
# 1. Učitavanje dataseta
dataset = load_dataset("ag_news")
print("Original dataset:", dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Original dataset: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [5]:
# 2. Smanjivanje dataseta na 10%
train_size = int(0.1 * len(dataset["train"]))
test_size = int(0.1 * len(dataset["test"]))

dataset["train"] = dataset["train"].select(range(train_size))
dataset["test"] = dataset["test"].select(range(test_size))

print("Reduced dataset:", dataset)

Reduced dataset: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 760
    })
})


In [6]:
train_dataset = dataset['train']
test_dataset = dataset['test']

In [7]:
# 3. Tokenizer i Model
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=4)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# 4. Tokenizacija Podataka
def preprocess_function(examples):
        return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
tokenized_test_dataset = tokenized_test_dataset.rename_column("label", "labels")

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

In [9]:
# 6. Konfiguracija metrika
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [10]:
# 7. Eksperimentisanje sa različitim vrednostima learning rate-a i batch size-a
learning_rates = [1e-5, 3e-5, 5e-5]
batch_sizes = [8, 16, 32]

best_f1 = 0
best_model = None
best_lr = None
best_bs = None

for lr in learning_rates:
    for bs in batch_sizes:
        print(f"\nTraining with learning rate: {lr}, batch size: {bs}")

        # 8. Resetovanje modela
        model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=4)

        # 9. TrainingArguments i Trainer
        training_args = TrainingArguments(
            output_dir=f"./results_tweet_eval_lr{lr}_bs{bs}",
            evaluation_strategy="epoch",
            learning_rate=lr,
            per_device_train_batch_size=bs,
            per_device_eval_batch_size=bs,
            num_train_epochs=3,
            weight_decay=0.01,
            save_strategy="no",
            push_to_hub=False,
        )

        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train_dataset,
            eval_dataset=tokenized_test_dataset,
            compute_metrics=compute_metrics
        )
        # 10. Treniranje modela
        trainer.train()

        # 11. Evaluacija modela
        results = trainer.evaluate()
        print(results)

        if results['eval_f1'] > best_f1:
            best_f1 = results['eval_f1']
            best_model = model
            best_lr = lr
            best_bs = bs
# 12. Prikaz najboljeg modela
print(f"\n Najbolji model sa learning rate: {best_lr}, batch size: {best_bs}, F1 score: {best_f1}")


Training with learning rate: 1e-05, batch size: 8


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.311600,0.306256,0.928947,0.928079,0.929559,0.928947
2,0.219700,0.297176,0.930263,0.930426,0.931100,0.930263
3,0.155400,0.306347,0.935526,0.935489,0.935539,0.935526


{'eval_loss': 0.30634692311286926, 'eval_accuracy': 0.9355263157894737, 'eval_f1': 0.9354892538040419, 'eval_precision': 0.9355385905634273, 'eval_recall': 0.9355263157894737, 'eval_runtime': 5.4092, 'eval_samples_per_second': 140.502, 'eval_steps_per_second': 17.563, 'epoch': 3.0}

Training with learning rate: 1e-05, batch size: 16


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.435700,0.249849,0.930263,0.929685,0.930386,0.930263
2,0.212200,0.237708,0.932895,0.933110,0.933763,0.932895
3,0.141000,0.250249,0.938158,0.938115,0.938115,0.938158


{'eval_loss': 0.25024881958961487, 'eval_accuracy': 0.9381578947368421, 'eval_f1': 0.9381150353716918, 'eval_precision': 0.9381153897704537, 'eval_recall': 0.9381578947368421, 'eval_runtime': 5.451, 'eval_samples_per_second': 139.425, 'eval_steps_per_second': 8.806, 'epoch': 3.0}

Training with learning rate: 5e-05, batch size: 32


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.286624,0.907895,0.907551,0.908927,0.907895
2,0.325200,0.222461,0.936842,0.936931,0.937571,0.936842
3,0.151600,0.233389,0.938158,0.938042,0.938182,0.938158


{'eval_loss': 0.2333887219429016, 'eval_accuracy': 0.9381578947368421, 'eval_f1': 0.9380415808963294, 'eval_precision': 0.9381818610425736, 'eval_recall': 0.9381578947368421, 'eval_runtime': 5.0199, 'eval_samples_per_second': 151.397, 'eval_steps_per_second': 4.781, 'epoch': 3.0}

 Najbolji model sa learning rate: 3e-05, batch size: 32, F1 score: 0.946117312608306
